### Social Network Graph
The first part of this project is modeling the social network over which we will simulate different events, namely parties, decay of relationships, falling outs, and meeting people.

We determined that the components of the graph are
* People: Each person is represented by a node
* Frendship / connection: Two connected people have an edge between them. The frequency of there being a connected is modeled off of Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships).
* Friendship level: The friendship level of these two people are represented as the weight of the edge. The specific friendship level is chosen using Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships). The specific break downs are:
    * Enemies (??): $<-1$ 
    * People you can recognize (1500): $\frac{0}{2215} \le x < \frac{1500}{2215}$.For most of our calculations, we will considert this range essentially 0.
    * Acquaintances (500): $\frac{1500}{2215} \le x < \frac{2000}{2215}$
    * Meaningful contacts (150): $\frac{2000}{2215} \le x < \frac{2150}{2215}$
    * Friends (50): $\frac{2150}{2215} \le x < \frac{2200}{2215}$
    * Good Friends (15): $\frac{2200}{2215} \le x < \frac{2215}{2215}$

In [10]:
import numpy as np
import math
from itertools import permutations, combinations

In [11]:
def generate_graph(n: int, p:float):
    # generate the graph parameters
    connections = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2)) > p
    weights = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2))

    # fill out actual graph
    graph = np.zeros((n, n))
    ui = (np.triu_indices(n)) # indices of the upper triangular matrix
    graph[ui] = weights
    graph[ui] = graph[ui] * connections# np.ma.masked_array(graph[ui], mask=connections) 

    # transpose edges for undirected graph
    graph = graph + np.transpose(graph[:, :])

    x = np.arange(n)

    graph[x, x] = 0 
    
    return graph

In [12]:
graph = generate_graph(4000, 1-2215/4000)
graph

array([[0.        , 0.        , 0.90893348, ..., 0.91608441, 0.        ,
        0.99920587],
       [0.        , 0.        , 0.98808688, ..., 0.        , 0.11167145,
        0.66512592],
       [0.90893348, 0.98808688, 0.        , ..., 0.        , 0.3679149 ,
        0.89721939],
       ...,
       [0.91608441, 0.        , 0.        , ..., 0.        , 0.49626238,
        0.58791338],
       [0.        , 0.11167145, 0.3679149 , ..., 0.49626238, 0.        ,
        0.0431956 ],
       [0.99920587, 0.66512592, 0.89721939, ..., 0.58791338, 0.0431956 ,
        0.        ]], shape=(4000, 4000))

In [13]:
# This function generates the social connections (direct and indirect) between some given person and all other people. In the instance where this person and another person
# are directly connected, the 'social connection' value will likely be equivalent to their current friendship level. Otherwise, indirect connects can have a social connection
# with the given person if they are typically w/in one degree, with a high level of friendship to the intermediary (some friends, and good friends. Occurs about 1% of the time)
# AND the intermediary has a high degree of friendship to the original person. A friend of a friend, so to speak.

def generate_social_connections(graph, person):
    # Build the initial social connections based off the person's direct network
    connectedness = graph[person].copy()
    
    # Note the blacklisted people
    black_list = np.where(connectedness < 0)[0]

    # Create a filter that removes blacklisted people
    mask = np.zeros(connectedness.size, dtype=bool)
    mask[black_list] = True
    mask[person] = True
    mask = np.vstack((mask, mask))
    

    # Apply the filter to connectedness, giving possible guests
    possible = np.ma.array(np.vstack((connectedness, np.array([1]*connectedness.shape[0]))), mask=mask)

        #print("step 1", possible) #testing
    
    # Continue below operations until the social connection value is arbitrarily low (within the lowest catgory)
    while (possible[0] >= 1500/2215).sum() > 0:

            #print("entered loop!") #testing
        
        # Choose the next highest social connection
        next_guest = np.argmax(possible[0])
        guest_distance = possible[1, next_guest]
        connectedness[next_guest] = possible[0, next_guest]

            #print("next guest:", next_guest, "distance:", guest_distance) #testing

        # Update filter to include the next added guest since we do not want to update this value.
        mask[:, next_guest] = True
        possible = np.ma.array(possible, mask=mask)
            #print("masked off next guest", possible) #testing

        
        # Improve social connections if a person is indirectly connected to the person
        updater = possible[0] < connectedness[next_guest] * graph[next_guest] / (guest_distance + 1)**(1/1.83)
            #print("updater", updater) # testing
        possible[1, updater] = guest_distance + 1
        possible[0, updater] = (connectedness[next_guest] * graph[next_guest] / (possible[1])**(1/1.83))[updater]

            #print("updated possible", possible) #testing
    
    return connectedness

In [14]:
test_graph = generate_graph(4000, 1-2215/4000)
connections = generate_social_connections(test_graph, 4)

print("Direct social connections vs. Total Social Connections")
print("Total edge weight:",sum(test_graph[4]), sum(connections))
print("Number of important edges(non-arbitrary connection):", sum(test_graph[4] > 1500/2215), sum(connections > 1500/2215))
print("Total number of edges:", sum(test_graph[4] > 0), sum(connections > 0))

print("\nGranular breakdown of direct and total social connections")
print("Number of acquaintances+:", sum((test_graph[4]) > 1500/2215),sum(connections > 1500/2215))
print("Number of meaningful contacts+:", sum(test_graph[4] > 2000/2215),sum(connections > 2000/2215))
print("Number of friends+:", sum(test_graph[4]>2150/2215), sum(connections > 2150/2215))
print("Number of good friends+:", sum(test_graph[4]>2200/2215),sum(connections > 2200/2215))

Direct social connections vs. Total Social Connections
Total edge weight: 1096.8350203083103 1262.2266063983732
Number of important edges(non-arbitrary connection): 712 1037
Total number of edges: 2184 2346

Granular breakdown of direct and total social connections
Number of acquaintances+: 712 1037
Number of meaningful contacts+: 235 235
Number of friends+: 78 78
Number of good friends+: 19 19


In [15]:
(1- ((1500/2215) * (2)**(1/1.83))) #Rough percentage of occurence

0.010957864640372073

### Invitations and Attending

In [16]:
def invite_random(graph, host):
    connections = generate_social_connections(graph, host)
    connections *= np.random.normal(loc=0.5, scale=0.33, size=graph.shape[0])
    return np.where(connections >= (np.random.uniform(low=0.33+.5, high=0.66+0.5))*2150/2215)[0]

In [17]:
def invite_threshold(graph, host, threshold = 2000/2215):
    return np.where(graph[host]>threshold)[0]

In [18]:
def attending(invitees, graph, host, base=0.5):
    attendees=[]

    mask = np.zeros(graph.shape[0], dtype=bool)
    mask[invitees] = True

    connections = generate_social_connections(graph, host)

    p_attend = base + 0.5 * (connections - 1500/2215)

    mask = mask & (np.random.uniform(0,1, graph.shape[0]) < p_attend)


    return np.where(mask)[0]

In [19]:
graph = generate_graph(4000, 1-2215/4000)
list1 = invite_threshold(graph, 4)
list2 = invite_random(graph, 4)

attending1 = attending(list1, graph, 4)
attending2 = attending(list2, graph, 4)

In [20]:
print(len(list1), len(attending1))
print(len(list2), len(attending2))

220 136
6 2


### Meeting People at a party

In [21]:
#Probability of meeting someone at the party
# I assume the probability of meeting someone at a party is proportional to the sum of all edge weights a person has
def prob_meeting(graph, person):
    total_weights=np.sum(graph) 
    person_weights = np.sum(graph[person])
    
    return person_weights/total_weights

# As we meet more people, the connection gets stronger and hence the weight increases by a factor j
def update_weight(graph, attendees, j=0.5):
    update_list = list(combinations(attendees, 2))

    print(update_list)
    for i in update_list:
        a,b = i

        if abs(graph[a,b]) > 0:
            graph[a,b] += j*(1-abs(graph[a,b]))
            graph[b,a]=graph[a,b] # This will keep the symmetry of the edge
        
        else:
            graph[a,b] = 1750/2215
            graph[b,a] = graph[a,b]
    
    return graph

In [36]:
graph = generate_graph(4000, 0.5)
list2 = invite_random(graph, 2)
# attending2 = attending(list2, graph, 2) 
attending2 = [0, 1, 2]
graph[[0,0, 1], [1,2, 2]]

array([0.25033565, 0.88097228, 0.90164448])

In [49]:
graph = update_weight(graph, attending2)
graph[[0,0, 1], [1,2, 2]]

[(0, 1), (0, 2), (1, 2)]


array([0.99990849, 0.99998547, 0.99998799])

### Decaying Friendships and Enemies

In [26]:
# This function will occur after a party happens
# Friendships will decay generally if two people were not in the same party together
# A select few random friendships will become enemies

def friendship_decay(graph, party: list[int]):
    # Get every pairing of people at the party
    party_pairs = list(permutations(party, 2))
    x, y = zip(*party_pairs)

    # Take note of the current values of the people who went to the party together
    sustained = graph[list(x), list(y)].copy()

    # Decrease friendship levels
    graph[(graph < 1500 /2215) & (graph > 0)] *= 0.60 # May need to play with these rates more to make it correct.
    graph[(graph < 2000 / 2215) & (graph >= 1500/2215)] *= 0.8
    graph[(graph < 2150 / 2215) & (graph >= 2000/2215)] *= 0.95
    graph[(graph < 2200 / 2215) & (graph >= 2150/2215)] *= 0.99
    graph[(graph <= 1) & (graph >= 2200/2215)] *= 0.997

    # Decrease enemy levels
    graph[(graph >= -1) & (graph < -2200/2215)] *= 0.997
    graph[(graph >= -2200/2215) & (graph < -2150/2215)] *= 0.99
    graph[(graph >= -2150 / 2215) & (graph < -2000/2215)] *= 0.95
    graph[(graph >= -2000/2215) & (graph < -1500/2215)] *= 0.8
    graph[(graph >= -1500 /2215) & (graph < 0)] *= 0.60

    # If the value is under 0.1, set to 0.
    graph[(abs(graph) < 0.1)] = 0

    # Reinstate the social connections level for people at the party
    graph[list(x), list(y)] = sustained

    return graph

def sudden_enemies(graph, num):
    n = graph.shape[0]
    i = 0
    while i < num:
        nodes = np.random.randint(0, high=n, size=(2))

        if nodes[0] == nodes[1]:
            i = i-1
        else:
            graph[nodes[0], nodes[1]] = -1
            graph[nodes[1], nodes[0]] = -1
        i += 1
    return graph

In [27]:
tester = generate_graph(4, 1-2215/4000)
tester = sudden_enemies(tester, 2)

tester

array([[ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.14250954],
       [-1.        , -1.        ,  0.        ,  0.54359242],
       [ 0.        ,  0.14250954,  0.54359242,  0.        ]])

In [28]:
tester = friendship_decay(tester, party=[0,1,2])
tester

array([[ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [-1.        , -1.        ,  0.        ,  0.32615545],
       [ 0.        ,  0.        ,  0.32615545,  0.        ]])

In [29]:
tester = np.ones((4,4)) * 0.9
tester

array([[0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9]])

In [30]:
tester = friendship_decay(tester, party=[1,2])
tester

array([[0.72, 0.72, 0.72, 0.72],
       [0.72, 0.72, 0.9 , 0.72],
       [0.72, 0.9 , 0.72, 0.72],
       [0.72, 0.72, 0.72, 0.72]])